# Imports

In [ ]:
import numpy as np
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
import pyarrow.feather as feather
import zarr
import dask
from dask import delayed
import distributed
from distributed import Client, LocalCluster, progress
from dask_jobqueue import SLURMCluster
import streamz
import streamz.dataframe as sdf
import holoviews as hv
from holoviews.streams import Stream, param, Selection1D
from holoviews.operation.datashader import regrid
from bokeh.models.tools import HoverTool, TapTool
import matplotlib.pyplot as plt
import qgrid
import ipywidgets as widgets
from tqdm import tnrange, tqdm, tqdm_notebook
import warnings
from functools import partial
from cytoolz import *
from operator import getitem
import nd2reader
from importlib import reload
import traceback
import hvplot.pandas
import param
import parambokeh
from traitlets import All
import cachetools
from collections import namedtuple, defaultdict
from collections.abc import Mapping, Sequence
from numbers import Number
import skimage.morphology
import scipy
from glob import glob
import asyncio
import time

IDX = pd.IndexSlice

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# from processing import *
# from trench_detection import *
# from trench_segmentation import *
# from trench_segmentation.watershed import *
# from util import *
# from ui import *
import common, trench_detection, util
import ui, diagnostics, metadata
import workflow, image, geometry
import trench_detection.hough, trench_detection.core
import trench_segmentation.watershed

In [ ]:
#%load_ext line_profiler
hv.extension("bokeh")
%matplotlib inline
tqdm.monitor_interval = 0
asyncio.get_event_loop().set_debug(True)
import logging

logging.basicConfig(level=logging.DEBUG)

# Restore data

In [ ]:
%store -r trench_points
%store -r trench_diag
%store -r trench_bboxes
trench_bboxes_t0 = util.get_one(trench_bboxes.groupby("t"))[1]

# Analysis

## Arrow

In [ ]:
arrow_filename = (
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.subset1000.arrow"
)

In [ ]:
columns = ["position", "label"]

In [ ]:
columns_to_drop = list(set(c.name for c in table0.columns) - set(columns))

In [ ]:
columns_to_drop

In [ ]:
table1 = table0.drop(columns_to_drop)

In [ ]:
reader = pa.open_stream(arrow_filename)
table0 = pa.Table.from_batches([reader.read_next_batch()])
imos = pa.BufferOutputStream()
buffer = imos.getvalue()
writer = pa.RecordBatchStreamWriter(buffer, table0)

In [ ]:
def read_arrow(arrow_filename, columns, categorical_columns=None, batch_size=1000):
    reader = pa.open_stream(arrow_filename)
    table0 = pa.Table.from_batches([reader.read_next_batch()])
    columns_to_drop = list(set(c.name for c in table0.columns) - set(columns))
    table1 = table0.drop(columns_to_drop)
    imos = pa.BufferOutputStream()
    writer = pa.RecordBatchStreamWriter(imos, table1.schema)
    t0 = time.time()
    for i, batches in enumerate(util.grouper(reader, batch_size)):
        if True:  # i % 100 == 0:
            t = time.time()
            dt = t - t0
            t0 = t
            print("batch", i, "time {:.2f}".format(dt))
        table = pa.Table.from_batches(batches).drop(columns_to_drop)
        print("    rows per second", len(table) / dt)
        writer.write_table(table)
    # with pq.ParquetWriter(parquet_filename, table0.schema) as writer:
    #     writer.write_table(table0)
    #    for batches in util.grouper(reader, batch_size):
    #         table = pa.Table.from_batches(batches)
    #         writer.write_table(table)
    output_reader = pa.open_stream(imos.getvalue())
    return output_reader

In [ ]:
# arrow_filename = '/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.subset1000.arrow'
# arrow_filename = '/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.subset15000.arrow'
# arrow_filename = '/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.subset15000.arrow'
arrow_filename = "/tmp/analysis_full_stream11_2.arrow"

In [ ]:
%%time
# cols = ['position', 'label', 'filename', 't', 'trench', 'trench_set', 'channel']
# cols = ["('YFP', 'labelwise', 'p0.9')"]
cols = [
    "filename",
    "position",
    "channel",
    "t",
    "trench_set",
    "trench",
    "label",
    "('YFP', 'labelwise', 'p0.9')",
    "('MCHERRY', 'labelwise', 'p0.9')",
    "('YFP', 'regionprops', 'area')",
]
b = read_arrow(arrow_filename, cols)

In [ ]:
%time table = b.read_all()

In [ ]:
len(table)

In [ ]:
%%time
tp = table.to_pandas(zero_copy_only=True)

In [ ]:
%%time
tp.info()  # memory_usage='deep')

In [ ]:
%%time
pq.write_table(
    table, "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.3cols.parquet"
)

In [ ]:
%%time
pq.write_table(
    table,
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.3cols.rg1000.parquet",
    row_group_size=1000,
)

In [ ]:
%%time
pq.write_table(
    table,
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.3cols.rg1000000.parquet",
    row_group_size=1000000,
)

In [ ]:
%%time
pq.write_table(
    table,
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.3cols.rg100000.parquet",
    row_group_size=100000,
)

In [ ]:
%%time
pq.write_table(
    pa.Table.from_pandas(tp),
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.3cols.roundtrip.parquet",
)

In [ ]:
pq.write_table?

In [ ]:
%%time
tp = pq.read_pandas(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.3cols.rg100000.parquet"
)  # .to_pandas()

In [ ]:
# parquet_filename = '/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.3cols.rg100000.parquet'
parquet_filename = "/tmp/analysis_full_stream11_2.3cols.rg100000.parquet"
f = pq.ParquetFile(parquet_filename)

In [ ]:
f.num_row_groups

In [ ]:
len(f.read_row_group(0))

In [ ]:
t = f.read_row_group(0)

In [ ]:
t

In [ ]:
%%time
t0 = time.time()
for i in range(f.num_row_groups):
    row_group = f.read_row_group(i)
    if i % 100 == 0:
        t = time.time()
        dt = t - t0
        t0 = t
        print("batch", i, "time {:.2f}".format(dt))

In [ ]:
%%time
tpq = pa.concat_tables([f.read_row_group(i) for i in range(f.num_row_groups)])

In [ ]:
c = tpq.column(3)

In [ ]:
tpq2 = tpq.set_column(3, tpq.column(3).dictionary_encode())

In [ ]:
%%time
pq.write_table(
    tpq2,
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.3cols.rg10000.dict.parquet",
    row_group_size=10000,
)

In [ ]:
pa.write

In [ ]:
%%time
c.dictionary_encode()

In [ ]:
%%time
tpqs = tpq.to_pandas(strings_to_categorical=True)

In [ ]:
tpqs

In [ ]:
x = pa.array(list("1" * 2**30))

demo = "demo.parquet"


def scenario():
    t = pa.Table.from_arrays([x], ["x"])
    writer = pq.ParquetWriter(demo, t.schema)
    for i in range(2):
        writer.write_table(t)
    writer.close()

    pf = pq.ParquetFile(demo)

    # pyarrow.lib.ArrowIOError: Arrow error: Invalid: BinaryArray cannot contain more than 2147483646 bytes, have 2147483647
    t2 = pf.read()

In [ ]:
scenario()

In [ ]:
%%time
f.read().to_pandas()

In [ ]:
f.read_row_group(10000).to_pandas()

In [ ]:
%%time
fastparquet.write(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.3cols.fastparquet",
    tp,
)

In [ ]:
tp

## Parquetify

In [ ]:
def parquetify(arrow_filename, parquet_filename=None, batch_size=1000):
    if parquet_filename is None:
        parquet_filename = arrow_filename.replace(".arrow", ".parquet")
    reader = pa.open_stream(arrow_filename)
    table0 = pa.Table.from_batches([reader.read_next_batch()])
    with pq.ParquetWriter(parquet_filename, table0.schema) as writer:
        writer.write_table(table0)
        for batches in util.grouper(reader, batch_size):
            table = pa.Table.from_batches(batches)
            writer.write_table(table)

In [ ]:
%%time
parquetify(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_0.arrow",
    parquet_filename="/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_0.test.parquet",
)

In [ ]:
z = pq.ParquetFile(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_1.parquet"
)

In [ ]:
z.num_row_groups

In [ ]:
%%time
pq.read_table(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_0.test.parquet"
).to_pandas()

In [ ]:
%%time
parquetify("/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_0.arrow")

In [ ]:
%%time
parquetify("/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_1.arrow")

In [ ]:
%%timeit
open(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_1.parquet", "rb"
).read()

In [ ]:
%%time
pq.read_table(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_1.parquet", nthreads=4
).to_pandas()

## Read Parquet from memory

In [ ]:
buf = pa.BufferOutputStream()

In [ ]:
in_file = pa.OSFile(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_1.arrow"
)

In [ ]:
%%time
buf.upload(in_file, buffer_size=2**20)

In [ ]:
bufr = pa.BufferReader(buf.getvalue())

In [ ]:
bufr.seek(0)

In [ ]:
%%time
t = pa.open_stream(bufr).read_all()

In [ ]:
in_file.seek(0)

In [ ]:
pa.MemoryMappedFile?

In [ ]:
in_file2 = pa.MemoryMappedFile(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_1.arrow"
)

In [ ]:
in_file2 = pa.memory_map(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_1.arrow"
)

In [ ]:
in_file2.seek(0)

In [ ]:
in_file2.read(100)

In [ ]:
%%time
tt = pa.open_stream(in_file2).read_all()

In [ ]:
t = pq.read_table(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_1.parquet", nthreads=1
)

In [ ]:
%%time
pq.write_table(
    t,
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_1.test.parquet",
    compression="zstd",
    row_group_size=10000,
    version="2.0",
)

In [ ]:
%%time
pq.read_table(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_1.test.parquet",
    nthreads=4,
)

In [ ]:
%prun pq.read_table('/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_1.parquet', nthreads=1)

In [ ]:
%prun pq.ParquetFile(bufr).read()

In [ ]:
%%time
pq.ParquetFile(bufr).read()

## Parquetify2

In [ ]:
def copy_arrow(in_filename, out_filename, length=None, batch_size=1000):
    in_file = pa.memory_map(in_filename)
    reader = pa.RecordBatchStreamReader(in_file)
    out_file = pa.OSFile(out_filename, "wb")
    table0 = pa.Table.from_batches([reader.read_next_batch()])
    writer = pa.RecordBatchStreamWriter(out_file, table0.schema)
    writer.write_table(table0)
    if length is not None:
        reader = take(length, reader)
    for batches in util.grouper(reader, batch_size):
        table = pa.Table.from_batches(batches)
        writer.write_table(table)

In [ ]:
%%time
copy_arrow(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.arrow",
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.subset.arrow",
    batch_size=1000,
    length=1000,
)

In [ ]:
%%time
copy_arrow(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.subset.arrow",
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.subset.rebatched1000.arrow",
    batch_size=1000,
    length=None,
)

In [ ]:
%%time
copy_arrow(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.subset.arrow",
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.subset.rebatched10000.arrow",
    batch_size=10000,
    length=None,
)

In [ ]:
%%time
copy_arrow(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.subset.arrow",
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.subset.rebatched100.arrow",
    batch_size=100,
    length=None,
)

In [ ]:
%%time
parquetify2(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.subset.arrow",
    batch_size=1,
    length=1000,
)

In [ ]:
%%time
parquetify2(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.subset.rebatched100.arrow",
    batch_size=1,
    length=1000,
)

In [ ]:
%%time
parquetify2(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.subset.rebatched1000.arrow",
    batch_size=1,
    length=1000,
)

In [ ]:
%%time
parquetify2(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.subset.rebatched10000.arrow",
    batch_size=1,
    length=1000,
)

In [ ]:
in_file = pa.OSFile(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.subset.arrow"
)

In [ ]:
reader = pa.RecordBatchStreamReader(in_file)

In [ ]:
batch = reader.read_next_batch()

In [ ]:
batches = list(take(10, reader))

In [ ]:
table = pa.Table.from_batches(batches)

In [ ]:
len(table)

In [ ]:
batches2 = table.to_batches(chunksize=10000)

In [ ]:
len(batches2)

In [ ]:
len(batches[0])

In [ ]:
len(batches2[0])

In [ ]:
col = table.column(0)

In [ ]:
col.data.

In [ ]:
pa.Table.from_batches([batches2[0]]).to_pandas().info(memory_usage="deep")

In [ ]:
len(batches2[0])

In [ ]:
len(batch)

In [ ]:
def parquetify2(arrow_filename, parquet_filename=None, batch_size=1000, length=None):
    if parquet_filename is None:
        parquet_filename = arrow_filename.replace(".arrow", ".parquet4")
    arrow_file = pa.OSFile(arrow_filename)
    # arrow_file = pa.memory_map(arrow_filename)
    # parquet_mmap = pa.memory_map(parquet_filename, 'wb')
    reader = pa.open_stream(arrow_file)
    table0 = pa.Table.from_batches([reader.read_next_batch()])
    if length is not None:
        reader = take(length, reader)
    with pq.ParquetWriter(parquet_filename, table0.schema) as writer:
        # with pq.ParquetWriter(parquet_mmap, table0.schema) as writer:
        writer.write_table(table0)
        for batches in util.grouper(reader, batch_size):
            table = pa.Table.from_batches(batches)
            writer.write_table(table)
    # arrow_file.close()
    # parquet_mmap.flush()
    # parquet_mmap.close()

In [ ]:
%%time
copy_arrow(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.arrow",
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.subset15000.arrow",
    length=15000,
    batch_size=1,
)

In [ ]:
%%time
parquetify2(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.arrow",
    batch_size=1000,
)

In [ ]:
cols = [
    "('YFP', 'labelwise', 'p0.9')",
    "('MCHERRY', 'labelwise', 'p0.9')",
    "('YFP', 'regionprops', 'mean')",
    "('YFP', 'regionprops', 'area')",
]

In [ ]:
# cols = ["('YFP', 'labelwise', 'p0.9')"]
# cols = ['position']
# cols = ["('YFP', 'labelwise', 'p0.9')", 'filename', 'position']
# cols = ["('YFP', 'labelwise', 'p0.9')", 'position', 'trench', 'label']
cols = ["('YFP', 'labelwise', 'p0.9')", "position", "trench", "label", "filename"]

In [ ]:
%%time
# reader = pq.ParquetFile('/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.parquet4')
reader = pq.ParquetFile(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.onecol.rg10000.parquet4"
)
# for i in range(0,reader.num_row_groups):
#     print('row group', i)
#     reader.read_row_group(i, use_pandas_metadata=True)
tables = [
    reader.read_row_group(i, columns=None, nthreads=1, use_pandas_metadata=False)
    for i in range(reader.num_row_groups)
]
table = pa.concat_tables(tables)
# t = table.to_pandas(use_threads=True, strings_to_categorical=True)

In [ ]:
import fastparquet

In [ ]:
%%time
tf = fastparquet.ParquetFile(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.parquet4"
)

In [ ]:
next(tf.iter_row_groups())

In [ ]:
tf.row_groups[0]

In [ ]:
tf.read_row_group(0, [("YFP", "labelwise", "p0.9")], {})

In [ ]:
%%time
tf = fastparquet.ParquetFile(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.parquet4"
)  # .to_pandas()

In [ ]:
tff = tf.to_pandas(columns=["position"])

In [ ]:
%%time
tf = fastparquet.ParquetFile(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.onecol.rg10000.parquet4"
).to_pandas()

In [ ]:
pd.read_parquet?

In [ ]:
%%time
reader = pq.ParquetFile(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.parquet4"
)
tables = [
    reader.read_row_group(i, columns=cols, nthreads=4, use_pandas_metadata=False)
    for i in range(reader.num_row_groups)
]
table = pa.concat_tables(tables)
# t = table.to_pandas(use_threads=True, strings_to_categorical=True)

In [ ]:
%%time
t = table.to_pandas(use_threads=True, strings_to_categorical=True)

In [ ]:
%%time
pq.write_table(
    pa.Table.from_pandas(t),
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.onecol.rg10000.parquet4",
    row_group_size=10000,
)

In [ ]:
%%time
tsel = pq.read_pandas(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.onecol.rg10000.parquet4"
).to_pandas()

In [ ]:
t.head()

In [ ]:
import sys

In [ ]:
len(t[("YFP", "labelwise", "p0.9")].data)

In [ ]:
t.info(memory_usage="deep")

In [ ]:
%%time
t2 = t.reset_index()

In [ ]:
%%time
t2.info(memory_usage="deep")

In [ ]:
%%time
t = pq.read_pandas(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.parquet4",
    nthreads=0,
    columns=cols,
)  # .to_pandas()

In [ ]:
%%time
tp = t.to_pandas(use_threads=True)

In [ ]:
tp.info(memory_usage="deep")

In [ ]:
reader = pq.ParquetFile(
    pa.OSFile("/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.parquet4")
)

In [ ]:
a = reader.read_row_group(0)

In [ ]:
a.schema

In [ ]:
len(t)

In [ ]:
%%time
tt = (
    pa.open_stream(
        pa.memory_map(
            "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.subset1000.arrow"
        )
    )
    .read_all()
    .to_pandas()
)

In [ ]:
%%time
ttt = (
    pa.open_stream(
        pa.OSFile(
            "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_2.subset1000.arrow"
        )
    )
    .read_all()
    .to_pandas()
)

In [ ]:
%%time
t.to_pandas()

In [ ]:
%%time
tt.to_pandas()

In [ ]:
tt.to_pandas?

In [ ]:
%prun parquetify2('/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_0.arrow')

## Load data

In [ ]:
%%time
framewise_df = pa.open_stream(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_0.arrow"
).read_pandas()

In [ ]:
framewise_df.info(memory_usage="deep")

In [ ]:
s = pa.open_stream("/n/scratch2/jqs1/fidelity/all/output/analysis50_stream_0.arrow")

In [ ]:
for a in take(2, s):
    print(a)

In [ ]:
s.read_next_batch?

In [ ]:
s.read_pandas()

In [ ]:
%prun pa.open_stream('/n/scratch2/jqs1/fidelity/all/output/analysis50_stream_0.arrow').read_all()#.read_pandas()

In [ ]:
%prun pa.open_stream('/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_0.arrow').read_all()#.read_pandas()

In [ ]:
%%time
trenchwise_df = pa.open_stream(
    "/n/scratch2/jqs1/fidelity/all/output/analysis_full_stream11_1.arrow"
).read_pandas()

In [ ]:
trenchwise_df.info(memory_usage="deep")

In [ ]:
%%time
labelwise_df = pq.read_pandas(
    "/n/scratch2/jqs1/fidelity/all/output/analysis50_full_2.parquet"
).to_pandas()

In [ ]:
labelwise_df.info(memory_usage="deep")

In [ ]:
# labelwise_df.index.names = ['filename', 'position', 't', 'trench_set', 'trench', 'label']
# labelwise_df.sort_index(inplace=True)

In [ ]:
len(labelwise_df)

In [ ]:
framewise_df.head()

In [ ]:
trenchwise_df.head()

In [ ]:
labelwise_df.head()

## Basic stats

In [ ]:
# labelwise_df2 = labelwise_df.copy()
labelwise_df.columns = ["_".join(col).strip() for col in labelwise_df.columns.values]

In [ ]:
labelwise_df.loc[:, "MCHERRY_regionprops_area"].plot(kind="hist", bins=100, log=False)

In [ ]:
selected_cells = labelwise_df2[
    (50 < labelwise_df2["MCHERRY_regionprops_area"])
    & (labelwise_df2["MCHERRY_regionprops_area"] < 300)
]

In [ ]:
selected_cells.loc[:, "MCHERRY_regionprops_area"].plot(kind="hist", bins=100, log=False)

In [ ]:
selected_cells.loc[:, "MCHERRY_labelwise_p0.5"].plot(kind="hist", bins=100, log=False)

In [ ]:
selected_cells.loc[:, "YFP_labelwise_p0.5"].plot(kind="hist", bins=100, log=True)

In [ ]:
labelwise_df[:10000].hvplot(
    x="MCHERRY_labelwise_p0.5", y="YFP_labelwise_p0.5", kind="scatter"
)

In [ ]:
yfp_cols = [c for c in labelwise_df.columns if c.startswith("YFP_labelwise_p")]
mcherry_cols = [c for c in labelwise_df.columns if c.startswith("MCHERRY_labelwise_p")]

In [ ]:
d = labelwise_df.loc[
    IDX["/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2", 2, :, 1, 50:60], yfp_cols
]

In [ ]:
d.hvplot.scatter("t", "YFP_labelwise_p0.95", by="trench")

In [ ]:
%%opts Scatter (size=3)
hv.Layout(
    [
        d.hvplot.scatter("t", c, by="trench", width=500, height=300)
        for c in reversed(yfp_cols)
    ]
).cols(1)

In [ ]:
d2 = labelwise_df.loc[
    IDX["/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2", 3, :, :, :], yfp_cols
]

In [ ]:
%%opts Scatter (size=3) [show_legend=False]
hv.Layout(
    [
        d2.hvplot.scatter("t", c, by="trench", width=500, height=300)
        for c in reversed(yfp_cols)
    ]
).cols(1)

In [ ]:
%%opts Scatter (size=3) [show_legend=False]
def get_plot(pos):
    c = "YFP_labelwise_p0.9"
    d = labelwise_df.loc[
        IDX["/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2", pos, :, :, :], c
    ]
    return d.hvplot.scatter("t", c, by="trench", width=500, height=300)


hv.HoloMap(
    {
        pos: get_plot(pos)
        for pos in labelwise_df.index._get_level_values(1, unique=True)[:20]
    }
)

In [ ]:
labelwise_df.head().loc[:, cols]

In [ ]:
d = labelwise_df.loc[
    IDX["/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2", 2, :, 1, 50:60], cols
]
m = pd.melt(d.loc[:, cols])
# m.index = d.index
# m

# Plot tapping

In [ ]:
LabelStream = ui.MultiIndexStream.define("LabelStream", labelwise_df.index)
label_stream = LabelStream()
box = ui.dataframe_browser(label_stream)
label_stream.event()
box

In [ ]:
p = d.hvplot.scatter("t", "YFP_labelwise_p0.95").options(tools=["tap", "hover"])
ui.selection_to_stream(p, label_stream)
p

In [ ]:
def cb(**kwargs):
    dat = labelwise_df.loc[
        workflow.stream_slice(
            labelwise_df.index.names, kwargs, t=slice(None), label=slice(None)
        ),
        :,
    ]
    plot = dat.hvplot.scatter("t", "YFP_labelwise_p0.95").options(
        tools=["tap", "hover"]
    )
    # ui.selection_to_stream(plot, label_stream)
    return plot


ui.viewer(cb, label_stream)

## Transcription errors

In [ ]:
selected_cells.head()

In [ ]:
selected_cells.index[0][:-1]

In [ ]:
trench_bboxes.head()

In [ ]:
selected_cells.head()

In [ ]:
selected_cells.index[0][2:-1]

In [ ]:
selected_trenches = trench_bboxes.xs(
    (*selected_cells.index[0][:2], "MCHERRY", 0, *selected_cells.index[0][3:-1]),
    drop_level=False,
)

In [ ]:
selected_trenches.head()

In [ ]:
workflow.get_trench_stacks(selected_trenches, all_frames, image_limits)

# Trench UI

In [ ]:
trenchwise_df.head(10)

In [ ]:
trench_bboxes.head(20)

In [ ]:
trench_bboxes[trench_bboxes[("info", "hough_value")] > 90].loc[
    IDX[:, :, ["MCHERRY"], 0, :, :], :
].head(10)

In [ ]:
trench_bboxes.head(10)

In [ ]:
selected_trenches_segmentation.head(10)

In [ ]:
# LabelStream = ui.MultiIndexStream.define('LabelStream', trench_bboxes.index)
LabelStream = ui.MultiIndexStream.define("LabelStream", labelwise_df.index)
# LabelStream = ui.MultiIndexStream.define('LabelStream', selected_trenches_segmentation.index)
label_stream = LabelStream()

box = ui.dataframe_browser(label_stream)
label_stream.event()
box

In [ ]:
label_stream.contents

In [ ]:
%%opts Layout [normalize=False]
%%opts Image [width=500 height=300]
# image_callback = partial(ui.get_trench_set_overlay, get_frame_func=workflow.get_nd2_frame_cached)
# p = ui.trench_set_viewer(trench_bboxes_t0, label_stream, channel='MCHERRY', image_callback=image_callback).options({'Bounds': dict(tools=['tap'])})
p = ui.get_trench_set_overlay(
    trench_bboxes_t0,
    channel="MCHERRY",
    **{
        k: label_stream.contents[k]
        for k in ("filename", "position", "t", "trench_set", "trench")
    },
)
p = p.options({"Bounds": dict(tools=["tap"])})
s = Selection1D(source=p)
p

In [ ]:
s

In [ ]:
s

In [ ]:
%%opts Layout [normalize=False]
%%opts Image [width=800 height=150]
image_callback = partial(
    ui.get_trench_set_overlay, get_frame_func=workflow.get_nd2_frame_cached
)
(
    ui.trench_set_viewer(
        trench_bboxes_t0, label_stream, channel="MCHERRY", image_callback=image_callback
    )
    + ui.trench_set_viewer(
        trench_bboxes_t0, label_stream, channel="YFP", image_callback=image_callback
    )
).cols(1)

In [ ]:
%%opts Layout [normalize=False]
%%output size=100
hover = HoverTool(
    tooltips=[
        ("(x,y)", "(@x{0[.]0}, @y{0[.]0})"),
        ("value", "@z"),
    ]
)
cb = compose(partial(ui.hover_image, hover), ui._trench_img, workflow.get_trench_image)
# cb = workflow.get_trench_image
(
    ui.trench_viewer(trench_bboxes, label_stream, channel="MCHERRY", image_callback=cb)
    + ui.trench_viewer(trench_bboxes, label_stream, channel="YFP", image_callback=cb)
).cols(1)

In [ ]:
import bokeh

In [ ]:
compose(partial(ui.hover_image, hover), ui._trench_img, workflow.get_trench_image)(
    trench_bboxes,
    "/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2",
    3,
    "MCHERRY",
    0,
    1,
    56,
)

In [ ]:
yfp = labelwise_df.loc[
    IDX["/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2", 3, 3, 1, 61],
    ("YFP", "labelwise", "mean"),
]

In [ ]:
yfp.plot.hist(bins=50)

In [ ]:
yfp2 = labelwise_df.loc[
    IDX["/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2", 3],
    ("YFP", "labelwise", "p0.95"),
]

In [ ]:
yfp2[yfp2 < 300].plot.hist(bins=50, log=True)

In [ ]:
yfp2[yfp2 > 200]

In [ ]:
yfp2.loc[IDX[:, 1, 60]]

In [ ]:
yfp2.swaplevel(0, 1).swaplevel(1, 2).sort_index()

In [ ]:
yfp2[yfp2 > 130].groupby(["trench_set", "trench"]).filter(lambda x: x.count() > 10)

In [ ]:
yfp[yfp > 130]

In [ ]:
labelwise_df.xs(
    IDX["/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2", 3, 2, 1, 56],
    drop_level=False,
)["YFP"]

In [ ]:
labelwise_df.loc[
    IDX["/n/scratch2/jqs1/fidelity/all/180405_txnerr002.nd2", 0, 2, 1, 56], "MCHERRY"
]

In [ ]:
plt.imshow(
    workflow.get_trench_image(
        trench_bboxes,
        channel="MCHERRY",
        **dissoc(label_stream.contents, "_df", "label"),
    )
)

In [ ]:
workflow.get_nd2_frame(
    channel="MCHERRY",
    **util.get_keys(label_stream.contents, "filename", "position", "t"),
)

In [ ]:
f = partial(util.get_keys, keys=["label"])
f(label_stream.contents)

In [ ]:
label_stream.contents.keys()

In [ ]:
ui.image_viewer(frame_stream)

In [ ]:
ui.show_frame_info(trench_diag, frame_stream)

In [ ]:
g = ui.show_grid(selected, stream=frame_stream)
g